# Projeto 2 - Ciência dos Dados

Nome: Gabriel Pascua de Freitas Moreira

Nome: Fernando Cesar Furtado Ballesteros Fincatti

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [198]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [199]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle
import itertools 

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [200]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano
#Conta: @gabrielpfm

#leitura do arquivo no formato JSON
#with open('auth.pass') as fp:    
 #   data = json.load(fp)

#Configurando a biblioteca. Não modificar
#auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
#auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [201]:
#Produto escolhido:
#produto = 'Nintendo'

#Quantidade mínima de mensagens capturadas:
#n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 450

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [202]:
#Cria um objeto para a captura
#api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang,tweet_mode="extended").items():    
    #msgs.append(msg.full_text.lower())
 #   i += 1
 # #  if i > n:
   #     break

#Embaralhando as mensagens para reduzir um possível viés
#shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [203]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
#if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
   # writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
  #  dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
   # dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

  #  dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
  #  dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
  #  writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [204]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [205]:
Excel = pd.read_excel("Nintendo.xlsx", sheet_name="Treinamento")


In [206]:
#Transformando Excel em 3 textos divididos por categoria

trein_cat = []
QTD_CAT = 3
for i in range(0,QTD_CAT):
    trein_cat.append(" ".join(Excel[Excel["Classificação"]==i].Treinamento))


In [207]:
Excel["Classificação"].value_counts(True)

1    0.615238
2    0.266667
0    0.118095
Name: Classificação, dtype: float64

In [208]:
#Três'textões' após a limpeza

cat_clean = []
serie = []
for i in range(0,QTD_CAT):
    cat_clean.append(cleanup(trein_cat[i]))
    serie.append(pd.Series(cat_clean[i].split()))

In [209]:
tudo = ""
for cat in cat_clean:
    tudo += cat
serie_tudo = pd.Series(tudo.split())

In [210]:
freq_abs = []
freq_rel = []
for i in range(0,QTD_CAT):
    freq_abs.append(serie[i].value_counts())
    freq_rel.append(serie[i].value_counts(True))

freq_abs_tudo = (serie_tudo.value_counts())
freq_rel_tudo = (serie_tudo.value_counts(True))


In [211]:
Probs = []
for i in range(0,QTD_CAT):
    Probs.append(freq_abs[i].sum()/freq_abs_tudo.sum())


In [212]:
set_=[]
for i in range(0,QTD_CAT):
    set_.append(set(freq_rel[i].index))
x = itertools.combinations(range(0,QTD_CAT),2)
inter = []
for i in x:
    i0=i[0]
    i1=i[1]
    inter.append(set_[i0].intersection(set_[i1]))


In [213]:
def Classificador(tweet,freq_abs,qtdcat,freq_ab,prob):
    categoria = 0
    maior_prob = 0
    Prob_Tweet_Categoria = {}
    Prob_Categoria_Tweet = {}
    for n in range(0,qtdcat):
        probabilidade = 1
        probtc = freq_abs[n][tweet.values]
        for palavra,quantidade in probtc.items():
            if np.isnan(quantidade) == True:
                laplace = 1/(len(freq_ab)+len(freq_abs[n]))
            else:
                laplace = (quantidade+1)/(len(freq_ab)+len(freq_abs[n]))
          
            probabilidade *= laplace 
            
        Prob_Tweet_Categoria[n] = probabilidade
    for n in range(0,qtdcat):
        prtoct = (Prob_Tweet_Categoria[n])*(prob[n])
        Prob_Categoria_Tweet[n] = prtoct
    for categorias, probab in Prob_Categoria_Tweet.items():
        if probab > maior_prob:
            maior_prob = probab
            categoria = categorias
    
    return categoria
        


In [214]:
#Entrada do Tweet
def Limpa_Tweet(Excel_Lido,Coluna,freq_rel,qtdcat,freq_ab,prob):
    indice = 0
    Excel_Lido.loc[:,"Classificação"] = 0
    for tweets in Excel_Lido.Teste:
        tweet_limpo = cleanup(tweets.lower())
        tweet = pd.Series(tweet_limpo.split())
        classificado = Classificador(tweet,freq_rel,qtdcat,freq_ab,prob)
        Excel_Lido.loc[indice,"Classificação"] = classificado
        indice +=1
    return Excel_Lido
    

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [215]:
Excel_Teste = pd.read_excel("Nintendo.xlsx", sheet_name="Teste")

Excel_Fixo = Excel_Teste

manual =[]
for classficacao in Excel_Teste["Classificação"]:
    manual.append(classficacao)
print(manual)



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [216]:
#3.448929531325091e-67
teste = "Teste"
Excel_Classificado = Limpa_Tweet(Excel_Teste,teste,freq_abs,QTD_CAT,freq_abs_tudo,Probs)
Excel_Classificado

,Teste,Classificação
0,rt @nintendeusa: os consoles da nintendo em ge...,1
1,@katgatito amo todos menos os gordao jgoador d...,1
2,como assim expansion pass nintendo puta que pa...,1
3,eu amo a nintendo mas do jeito mercenário que ...,0
4,"rt @mandsmors: mário travou o nintendo, que ro...",1
5,jogabilidade que é bom 🏊‍♂️ \nparece joguinho ...,1
6,cade meu dk2 dona nintendo? https://t.co/hn8fo...,1
7,@__1089__ @0tw0__ @readnowayu1 @rogue_ii é uma...,1
8,"@midiasonysta @necallivaldo exatamente mídia,e...",1
9,"@nintendoes nintendo, dos décadas desfasados.",1


In [217]:
classificador = []
for classficacao in Excel_Classificado["Classificação"]:
    classificador.append(classficacao)
print(classificador)

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [218]:
Teste_de_Performance = {"Total":{"Certo":0,"Errado":0},0:{"Certo":0,"Errado":0},1:{"Certo":0,"Errado":0},2:{"Certo":0,"Errado":0}}

for i in range(len(manual)):
    if classificador[i] == manual[i]:
        Teste_de_Performance[manual[i]]["Certo"]+=1
        Teste_de_Performance["Total"]["Certo"]+=1
    else:
        Teste_de_Performance[manual[i]]["Errado"]+=1
        Teste_de_Performance["Total"]["Errado"]+=1
            
    
    
print(Teste_de_Performance)
Porcentagem_de_Acerto ={}

for categoria in Teste_de_Performance:
    if Teste_de_Performance[categoria]["Certo"]!=0 and Teste_de_Performance[categoria]["Certo"]!=0:
        Porcentagem_de_Acerto[categoria] = Teste_de_Performance[categoria]["Certo"]/(Teste_de_Performance[categoria]["Certo"]+Teste_de_Performance[categoria]["Errado"])
print(Porcentagem_de_Acerto)


{'Total': {'Certo': 251, 'Errado': 149}, 0: {'Certo': 1, 'Errado': 46}, 1: {'Certo': 230, 'Errado': 8}, 2: {'Certo': 20, 'Errado': 95}}
{'Total': 0.6275, 0: 0.02127659574468085, 1: 0.9663865546218487, 2: 0.17391304347826086}


## Segunda tentativa da criação do classificador
Devido a problemas na distribuição da classificação dos tweets tanto na base de treinamento quanto no teste, foi necessário juntar duas categorias, a categoria 0 e 2, que eram respectivamente positivos e negativos serão agrupadas em um categoria que será opiniões, numerada com 0, enquanto a categoria 1 continuará sendo neutros

In [219]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [220]:
Excel2 = pd.read_excel("Nintendo2.xlsx", sheet_name="Treinamento")


In [221]:
#Transformando Excel em 3 textos divididos por categoria

trein_cat2 = []
QTD_CAT2 = 2
for i in range(0,QTD_CAT2):
    trein_cat2.append(" ".join(Excel2[Excel2["Classificação"]==i].Treinamento))


In [222]:
Excel2["Classificação"].value_counts(True)

1    0.597774
0    0.402226
Name: Classificação, dtype: float64

In [223]:
#'textões' após a limpeza

cat_clean2 = []
serie2 = []
for i in range(0,QTD_CAT2):
    cat_clean2.append(cleanup(trein_cat2[i]))
    serie2.append(pd.Series(cat_clean2[i].split()))

In [224]:
tudo2 = ""
for cat in cat_clean2:
    tudo2 += cat
serie_tudo2 = pd.Series(tudo2.split())


In [225]:
freq_abs2 = []
freq_rel2 = []
for i in range(0,QTD_CAT2):
    freq_abs2.append(serie2[i].value_counts())
    freq_rel2.append(serie2[i].value_counts(True))

freq_abs_tudo2 = (serie_tudo2.value_counts())
freq_rel_tudo2 = (serie_tudo2.value_counts(True))


In [226]:
Probs2 = []
for i in range(0,QTD_CAT2):
    Probs2.append(freq_abs2[i].sum()/freq_abs_tudo2.sum())


In [227]:
set_2=[]
for i in range(0,QTD_CAT2):
    set_2.append(set(freq_rel2[i].index))
x = itertools.combinations(range(0,QTD_CAT2),2)
inter2 = []
for i in x:
    i02=i[0]
    i12=i[1]
    inter2.append(set_2[i02].intersection(set_2[i12]))


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [228]:
Excel_Teste2 = pd.read_excel("Nintendo2.xlsx", sheet_name="Teste")



manual2 =[]
for classficacao in Excel_Teste2["Classificação"]:
    manual2.append(classficacao)


teste = "Teste"
Excel_Classificado2 = Limpa_Tweet(Excel_Teste2,teste,freq_abs2,QTD_CAT2,freq_abs_tudo2,Probs2)
Excel_Classificado2;


In [229]:
classificador2 = []
for classficacao in Excel_Classificado2["Classificação"]:
    classificador2.append(classficacao)
print(classificador2)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 

In [230]:
Teste_de_Performance2 = {"Total":{"Certo":0,"Errado":0},0:{"Certo":0,"Errado":0},1:{"Certo":0,"Errado":0}}

for i in range(len(manual2)):
    if classificador2[i] == manual2[i]:
        Teste_de_Performance2[manual2[i]]["Certo"]+=1
        Teste_de_Performance2["Total"]["Certo"]+=1
    else:
        Teste_de_Performance2[manual2[i]]["Errado"]+=1
        Teste_de_Performance2["Total"]["Errado"]+=1
            
    
    
print(Teste_de_Performance2)
Porcentagem_de_Acerto2 ={}

for categoria in Teste_de_Performance2:
    if Teste_de_Performance2[categoria]["Certo"]!=0 and Teste_de_Performance2[categoria]["Certo"]!=0:
        Porcentagem_de_Acerto2[categoria] = Teste_de_Performance2[categoria]["Certo"]/(Teste_de_Performance2[categoria]["Certo"]+Teste_de_Performance2[categoria]["Errado"])
print(Porcentagem_de_Acerto2)


{'Total': {'Certo': 298, 'Errado': 108}, 0: {'Certo': 85, 'Errado': 81}, 1: {'Certo': 213, 'Errado': 27}}
{'Total': 0.7339901477832512, 0: 0.5120481927710844, 1: 0.8875}


## Tentativa com quatro categorias

In [220]:
Excel3 = pd.read_excel("Nintendo3.xlsx", sheet_name="Treinamento")


In [221]:
#Transformando Excel em 3 textos divididos por categoria

trein_cat3 = []
QTD_CAT3 = 4
for i in range(0,QTD_CAT3):
    trein_cat3.append(" ".join(Excel3[Excel3["Classificação"]==i].Treinamento))


In [222]:
Excel3["Classificação"].value_counts(True)

1    0.597774
0    0.402226
Name: Classificação, dtype: float64

In [223]:
#'textões' após a limpeza

cat_clean3 = []
serie3 = []
for i in range(0,QTD_CAT3):
    cat_clean3.append(cleanup(trein_cat3[i]))
    serie3.append(pd.Series(cat_clean3[i].split()))

In [224]:
tudo3 = ""
for cat in cat_clean3:
    tudo3 += cat
serie_tudo3 = pd.Series(tudo3.split())


In [225]:
freq_abs3 = []
freq_rel3 = []
for i in range(0,QTD_CAT3):
    freq_abs3.append(serie3[i].value_counts())
    freq_rel3.append(serie3[i].value_counts(True))

freq_abs_tudo3 = (serie_tudo3.value_counts())
freq_rel_tudo3 = (serie_tudo3.value_counts(True))


In [226]:
Probs3 = []
for i in range(0,QTD_CAT3):
    Probs3.append(freq_abs3[i].sum()/freq_abs_tudo3.sum())


In [227]:
set_3=[]
for i in range(0,QTD_CAT3):
    set_3.append(set(freq_rel3[i].index))
x = itertools.combinations(range(0,QTD_CAT3),2)
inter3 = []
for i in x:
    i03=i[0]
    i13=i[1]
    inter3.append(set_3[i03].intersection(set_3[i13]))


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [228]:
Excel_Teste3 = pd.read_excel("Nintendo2.xlsx", sheet_name="Teste")



manual3 =[]
for classficacao in Excel_Teste3["Classificação"]:
    manual3.append(classficacao)


teste = "Teste"
Excel_Classificado3 = Limpa_Tweet(Excel_Teste3,teste,freq_abs3,QTD_CAT3,freq_abs_tudo3,Probs3)
Excel_Classificado3;


In [229]:
classificador3 = []
for classficacao in Excel_Classificado3["Classificação"]:
    classificador3.append(classficacao)
print(classificador3)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 

In [230]:
Teste_de_Performance3 = {"Total":{"Certo":0,"Errado":0},0:{"Certo":0,"Errado":0},1:{"Certo":0,"Errado":0}}

for i in range(len(manual3)):
    if classificador3[i] == manual3[i]:
        Teste_de_Performance3[manual3[i]]["Certo"]+=1
        Teste_de_Performance3["Total"]["Certo"]+=1
    else:
        Teste_de_Performance3[manual3[i]]["Errado"]+=1
        Teste_de_Performance3["Total"]["Errado"]+=1
            
    
    
print(Teste_de_Performance3)
Porcentagem_de_Acerto3 ={}

for categoria in Teste_de_Performance3:
    if Teste_de_Performance3[categoria]["Certo"]!=0 and Teste_de_Performance3[categoria]["Certo"]!=0:
        Porcentagem_de_Acerto3[categoria] = Teste_de_Performance3[categoria]["Certo"]/(Teste_de_Performance3[categoria]["Certo"]+Teste_de_Performance2[categoria]["Errado"])
print(Porcentagem_de_Acerto3)


{'Total': {'Certo': 298, 'Errado': 108}, 0: {'Certo': 85, 'Errado': 81}, 1: {'Certo': 213, 'Errado': 27}}
{'Total': 0.7339901477832512, 0: 0.5120481927710844, 1: 0.8875}


___
### Concluindo

Observando os percentuais de acerto obtidos podemos concluir que o classificador funciona bem para a categoria 1 e a categoria 2, neutro e positivo, respectivamente, enquanto para a categoria 0, negativo, ele ainda apresenta dificuldade em classificar.

Isso se deve ao tamanho da base de dados que ele possui, onde  os tweets são classificados como neutros majoritariamente, mostrando assim a tendência do tipo de tweets feito sobre a empresa "Nintendo"  

### Porque não podemos utilizar o próprio classificador para gerar mais amostras de treinamento?
Essa é uma pergunta que pode surgir quando se está fazendo um classificador, e apesar disso parecer uma boa ideia, esse método pode estragar seu classificador, uma vez que o intuito de gerar mais amostra de treinamento é justamente melhorar o classificador, uma vez que ele não é perfeito e pode cometer erros.

Se o classificador for utilizado para gerar mais amostras, ele vai acabar repetindo os seus erros mais comuns e ficar cada vez mais longe de ser um bom classificador, o processo de geração de amostras de treinamento deve ser feito manualmente, assim deixando o classificador cada vez mais eficiente

### Propondo diferentes cenários para o Naive Bayes

Como proposta de outras situações possíveis para o uso do Naive Bayes, poderiamos citar por exemplo, a classificação de posts no facebook por uma empresa de eventos, a fim de receber feedbacks em relação ao útimo show realizado pela companhia. Assim poderia propor melhorias e aperfeiçoar o serviço, atraindo mais clientes e portanto, mais capital. 

Outro cenário possível seria a utilização do Naive Bayes para classifcar as respostas de um pequeno formulário preenchido pelos clientes do restaurante a respeito do atendimento, assim o gerente conseguiria saber a satisfação de seus clientes, e tomar decisões como por exemplo, demitir seus funcionários.




### Sugerindo e explicando possíveis melhorias para o projeto

Uma melhoria que poderia ajudar o nosso classificador a aumentar a taxa de acerto seria elevar o número de tweets presentes na base, além de manter uma distribuição proporcional entre as categorias nas respectivas bases, treinamento e teste.

Outra melhoria possível seria ensinar o classificador a classificar emojis da seguinte maneira:
    1 - Fariamos uma lista de emojis; 
    2 - Classificariamos os principais emojis em relação as categorias escolhidas;
    3 - Fariamos uma função para que, durante a classificação dos tweets, o classificador consultasse a lista de emojis e colocasse o resultado de acordo com a mesma.

Utilizar algum software ou programa terceirizado para melhorar a classificação dos tweets presentes na base.


## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**